In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FirstSparkSessionApp").getOrCreate()

24/12/06 15:20:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 15:20:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/06 15:20:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [26]:
df = spark.read.format('json').load('data/2015-summary.json', inferSchema=True, header=True )

In [27]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [28]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [7]:
df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [29]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [9]:
#spark DataTable
df.select("DEST_COUNTRY_NAME").show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [10]:
df.createOrReplaceTempView("mobility_data")

In [14]:
df.count()

256

In [30]:
df_dup2= df.select('DEST_COUNTRY_NAME').dropDuplicates()
df_dup2.show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [31]:
df_dup3= df.select('DEST_COUNTRY_NAME').dropDuplicates().cache()
df_dup3.show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [35]:
df_dup.count()

132

In [36]:
df_dup.distinct().count()

132

In [37]:
df.sort('DEST_COUNTRY_NAME').show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Algeria|      United States|    4|
|              Angola|      United States|   15|
|            Anguilla|      United States|   41|
| Antigua and Barbuda|      United States|  126|
|           Argentina|      United States|  180|
|               Aruba|      United States|  346|
|           Australia|      United States|  329|
|             Austria|      United States|   62|
|          Azerbaijan|      United States|   21|
|             Bahrain|      United States|   19|
|            Barbados|      United States|  154|
|             Belgium|      United States|  259|
|              Belize|      United States|  188|
|             Bermuda|      United States|  183|
|             Bolivia|      United States|   30|
|Bonaire, Sint Eus...|      United States|   58|
|              Brazil|      United States|  853|
|British Virgin Is..

In [19]:
from pyspark.sql.functions import expr

In [22]:
df3= df.withColumn('withInCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME'))

In [38]:
df3.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withInCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [23]:
#SQL 구문 CASE WHEN > 수치형 변수 > 명목형 변수로 변환 
df4= df.withColumn('category', expr('CASE WHEN count<10 THEN "under" WHEN count>=10 THEN "upper" END'))

In [24]:
df4.show(5)

+-----------------+-------------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|
+-----------------+-------------------+-----+--------+
|    United States|            Romania|   15|   upper|
|    United States|            Croatia|    1|   under|
|    United States|            Ireland|  344|   upper|
|            Egypt|      United States|   15|   upper|
|    United States|              India|   62|   upper|
+-----------------+-------------------+-----+--------+
only showing top 5 rows



In [ ]:
#count 2배 

In [39]:
df5 = df4.withColumn('withInCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME'))

In [40]:
df5.show()

+--------------------+-------------------+-----+--------+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withInCountry|
+--------------------+-------------------+-----+--------+-------------+
|       United States|            Romania|   15|   upper|        false|
|       United States|            Croatia|    1|   under|        false|
|       United States|            Ireland|  344|   upper|        false|
|               Egypt|      United States|   15|   upper|        false|
|       United States|              India|   62|   upper|        false|
|       United States|          Singapore|    1|   under|        false|
|       United States|            Grenada|   62|   upper|        false|
|          Costa Rica|      United States|  588|   upper|        false|
|             Senegal|      United States|   40|   upper|        false|
|             Moldova|      United States|    1|   under|        false|
|       United States|       Sint Maarten|  325|   upper|       

In [ ]:
#groupby 집계  

In [42]:
df5.groupBy("withInCountry").count().show()

+-------------+-----+
|withInCountry|count|
+-------------+-----+
|         true|    1|
|        false|  255|
+-------------+-----+



In [41]:
df5.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|   under|   48|
|   upper|  208|
+--------+-----+



# Projection 과 Filter
### select a,b,c from TableA  # projection > column > Transformation
### where a>10  #filter > Row >Transformation

In [43]:
df6 = df5.where('count<5' )

In [44]:
df6.show()

+--------------------+-------------------+-----+--------+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withInCountry|
+--------------------+-------------------+-----+--------+-------------+
|       United States|            Croatia|    1|   under|        false|
|       United States|          Singapore|    1|   under|        false|
|             Moldova|      United States|    1|   under|        false|
|               Malta|      United States|    1|   under|        false|
|             Algeria|      United States|    4|   under|        false|
|       United States|          Gibraltar|    1|   under|        false|
|Saint Vincent and...|      United States|    1|   under|        false|
|            Suriname|      United States|    1|   under|        false|
|       United States|             Cyprus|    1|   under|        false|
|       United States|           Malaysia|    3|   under|        false|
|            Thailand|      United States|    3|   under|       

In [45]:
df6.count()

46

In [ ]:
where a>10 and b!=1

In [46]:
df7 = df5.where('count<5' ).where('ORIGIN_COUNTRY_NAME != "United States"')
df7.show()

+-----------------+-------------------+-----+--------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withInCountry|
+-----------------+-------------------+-----+--------+-------------+
|    United States|            Croatia|    1|   under|        false|
|    United States|          Singapore|    1|   under|        false|
|    United States|          Gibraltar|    1|   under|        false|
|    United States|             Cyprus|    1|   under|        false|
|    United States|           Malaysia|    3|   under|        false|
|    United States|            Vietnam|    2|   under|        false|
|    United States|            Estonia|    1|   under|        false|
|    United States|            Hungary|    3|   under|        false|
|    United States|           Thailand|    4|   under|        false|
|    United States|            Liberia|    2|   under|        false|
|    United States|              Malta|    2|   under|        false|
|    United States|          Lithu

In [ ]:
#프로젝션, 필터링 연습

In [ ]:
#국내 여행이 아니면서 가장 횟수가 많은 ORIGIN_COUNTRY_NAME top 10을  추출해보세요

In [ ]:
#국내 여행이 아니면서 가장 횟수가 적은 ORIGIN_COUNTRY_NAME top 10을  추출해보세요

In [ ]:
#도착국가별 count 총합이 가장 많은 top10을 추출해 보세요